In [661]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import openpyxl
import itertools
from sklearn.feature_extraction.text import CountVectorizer

In [1]:
chat_words_str = """
DM=Direct message
CT=Cuttweet
RT=Retweet
PRT=Partial retweet
MT=Modified tweet
PRT=Please retweet
HT=Hat tip
CC=Carbon-copy
CX=Correction
EM=Email Marketing
SEO=Search Engine Optimization
SROI=Social Return on Investment
SN=Social Network
YT=YouTube
UGC=User-Generated Content
SMO=Social Media Optimization
FB=Facebook
LI=LinkedIn
SM=Social Media
SMM=Social Media Marketing
EZine=Electronic Magazine
BGD=Background
CD9=Code 9, parents are around
BTW=By the way
AB=About
ABT=About
DD=Dear daughter
AFAIK=far as I know
AYFKMWTS=Are you f—ing kidding me with this s—?
BR=Best regards
CHK=Check
CUL8R=See you later
DP=used to mean “profile pic”
FML=F— my life
FUBAR=F—ed up beyond all repair (slang from the US Military)
BBFN=Bye for now
B4=Before
DS=Dear son
FF=Follow Friday
EMA=Email address
DYK=Do you know
F2F=Face to face
FTF=Face to face
HAGN=Have a good night
DF=Dear fiancé
DAM=Don’t annoy me
FFS=For F—‘s Sake
EM=Email
EML=Email
FOTD=Find of the day
FTW=For the win, F— the world
FWIW=For what it’s worth
HTH=Hope that helps
GMAFB=Give me a f—ing break
HAND=Have a nice day
ICYMI=In case you missed it
GTFOOH=Get the f— out of here
GTS=Guess the song
HOTD=Headline of the day
IIRC=If I remember correctly
KYSO=Knock your socks off
KK=Kewl kewl, or ok, got it
HT=Head through
IC=I see
IDK=I don’t know
LHH=hella hard
ZOMG=OMG to the max
IMHO=In my humble opinion
NFW=No f—ing way
ORLY=Oh, really?
YOYO=You’re on your own
LMAO=Laughing my ass off
IRL=In real life
JK=Just kidding
JV=Joint venture
LO=Little One
LOL=Laugh out loud
MM=Music Monday
LMK=Let me know
TY=Thank you
SRS=Serious
STF=Shut the f—
STFU=Shut the f— up!
TL=Timeline
TYIA=Thank you in advance
TT=Trending topic
TYVW=Thank you very much
BRB=Be Right Back
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
SFW=Safe for work
TY=Thank you
TMB=Tweet me back
BRB=Be Right Back
IMO=In My Opinion
RLRT=Real-life re-tweet, a close cousin to OH
OOMF=One of my friends/followers
NTS=Note to self
RTFM=Read the f—ing manual
SNAFU=Situation normal, all f—ed up (slang from the US Military)
RLRT=Real-life re-tweet, a close cousin to OH
SMH=Shaking my head
STFW=Search the f—ing web!
TFTT=Thanks for this tweet
SOB=Son of a B—-
TFTF=Thanks for the follow
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

# Data preprocessing

In [663]:
df = pd.read_excel("template.xlsx")

## Chat Words Conversion

In [664]:
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)


def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)


df["text_chat_conversion"] = df["text"].apply(lambda text: correct_spellings(text))
df

,text,text_chat_conversion
0,"The cars drive, fast and erratically",The cars drive fast and erratically
1,the car dsrive slow?,the car drive slow
2,The bike ride very fast and erratically!,The bike ride very fast and erratically


## Spelling correction

In [665]:
from spellchecker import SpellChecker

spell = SpellChecker()


def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)


df["correct_spellings"] = df["text_chat_conversion"].apply(lambda text: correct_spellings(text))

## Punctuation removal

In [666]:
PUNCT_TO_REMOVE = string.punctuation


def remove_punctation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))


df["text_punctation_remove"] = df["correct_spellings"].apply(lambda text: remove_punctation(text))
df["text_lower"] = df["text_punctation_remove"].str.lower()

## Stop Words removal

In [667]:
from nltk.corpus import stopwords

", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [668]:
STOPWORDS = set(stopwords.words('english'))


def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


df["text_stopwords_removal"] = df["text_lower"].apply(lambda text: remove_stopwords(text))

## Freq and rare words

In [669]:
# from collections import Counter
# cnt = Counter()
# for text in df["text_stopwords_removal"].values:
#     for word in text.split():
#         cnt[word] += 1
#
# cnt.most_common(10)

In [670]:
# FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
# def remove_freqwords(text):
#     """custom function to remove the frequent words"""
#     return " ".join([word for word in str(text).split() if word not in FREQWORDS])
#
# df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
# df.head()

In [671]:
# n_rare_words = 10
# RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
# def remove_rarewords(text):
#     """custom function to remove the rare words"""
#     return " ".join([word for word in str(text).split() if word not in RAREWORDS])
#
# df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
# df.head()

## Stemming

In [672]:
from nltk.stem.porter import PorterStemmer

# Drop the two columns
stemmer = PorterStemmer()


def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])


df["text_stemmed"] = df["text_stopwords_removal"].apply(lambda text: stem_words(text))

In [673]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


df["text_preprocessed"] = df["text_stopwords_removal"].apply(lambda text: lemmatize_words(text))


In [674]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N": wordnet.NOUN, "V": wordnet.VERB, "J": wordnet.ADJ, "R": wordnet.ADV}


def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join(
        [lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])


df["text_lemmatized_with_pos"] = df["correct_spellings"].apply(lambda text: lemmatize_words(text))
df.head()


,text,text_chat_conversion,correct_spellings,text_punctation_remove,text_lower,text_stopwords_removal,text_stemmed,text_preprocessed,text_lemmatized_with_pos
0,"The cars drive, fast and erratically",The cars drive fast and erratically,The cars drive fast and erratically,The cars drive fast and erratically,the cars drive fast and erratically,cars drive fast erratically,car drive fast errat,car drive fast erratically,The car drive fast and erratically
1,the car dsrive slow?,the car drive slow,the car drive slow,the car drive slow,the car drive slow,car drive slow,car drive slow,car drive slow,the car drive slow
2,The bike ride very fast and erratically!,The bike ride very fast and erratically,The bike ride very fast and erratically,The bike ride very fast and erratically,the bike ride very fast and erratically,bike ride fast erratically,bike ride fast errat,bike ride fast erratically,The bike ride very fast and erratically


In [675]:
## Emoji and emoticons removal/conversion

In [676]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
# def remove_emoji(string):
#     emoji_pattern = re.compile("["
#                            u"\U0001F600-\U0001F64F"  # emoticons
#                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            u"\U00002702-\U000027B0"
#                            u"\U000024C2-\U0001F251"
#                            "]+", flags=re.UNICODE)
#     return emoji_pattern.sub(r'', string)
# df["text_no_emoji"] = df["text_lemmatized_with_pos"].apply(lambda text: remove_emoji(text))

In [677]:
# Thanks : https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py
EMOTICONS = {
    u":‑\)": "Happy face or smiley",
    u":\)": "Happy face or smiley",
    u":-\]": "Happy face or smiley",
    u":\]": "Happy face or smiley",
    u":-3": "Happy face smiley",
    u":3": "Happy face smiley",
    u":->": "Happy face smiley",
    u":>": "Happy face smiley",
    u"8-\)": "Happy face smiley",
    u":o\)": "Happy face smiley",
    u":-\}": "Happy face smiley",
    u":\}": "Happy face smiley",
    u":-\)": "Happy face smiley",
    u":c\)": "Happy face smiley",
    u":\^\)": "Happy face smiley",
    u"=\]": "Happy face smiley",
    u"=\)": "Happy face smiley",
    u":‑D": "Laughing, big grin or laugh with glasses",
    u":D": "Laughing, big grin or laugh with glasses",
    u"8‑D": "Laughing, big grin or laugh with glasses",
    u"8D": "Laughing, big grin or laugh with glasses",
    u"X‑D": "Laughing, big grin or laugh with glasses",
    u"XD": "Laughing, big grin or laugh with glasses",
    u"=D": "Laughing, big grin or laugh with glasses",
    u"=3": "Laughing, big grin or laugh with glasses",
    u"B\^D": "Laughing, big grin or laugh with glasses",
    u":-\)\)": "Very happy",
    u":‑\(": "Frown, sad, andry or pouting",
    u":-\(": "Frown, sad, andry or pouting",
    u":\(": "Frown, sad, andry or pouting",
    u":‑c": "Frown, sad, andry or pouting",
    u":c": "Frown, sad, andry or pouting",
    u":‑<": "Frown, sad, andry or pouting",
    u":<": "Frown, sad, andry or pouting",
    u":‑\[": "Frown, sad, andry or pouting",
    u":\[": "Frown, sad, andry or pouting",
    u":-\|\|": "Frown, sad, andry or pouting",
    u">:\[": "Frown, sad, andry or pouting",
    u":\{": "Frown, sad, andry or pouting",
    u":@": "Frown, sad, andry or pouting",
    u">:\(": "Frown, sad, andry or pouting",
    u":'‑\(": "Crying",
    u":'\(": "Crying",
    u":'‑\)": "Tears of happiness",
    u":'\)": "Tears of happiness",
    u"D‑':": "Horror",
    u"D:<": "Disgust",
    u"D:": "Sadness",
    u"D8": "Great dismay",
    u"D;": "Great dismay",
    u"D=": "Great dismay",
    u"DX": "Great dismay",
    u":‑O": "Surprise",
    u":O": "Surprise",
    u":‑o": "Surprise",
    u":o": "Surprise",
    u":-0": "Shock",
    u"8‑0": "Yawn",
    u">:O": "Yawn",
    u":-\*": "Kiss",
    u":\*": "Kiss",
    u":X": "Kiss",
    u";‑\)": "Wink or smirk",
    u";\)": "Wink or smirk",
    u"\*-\)": "Wink or smirk",
    u"\*\)": "Wink or smirk",
    u";‑\]": "Wink or smirk",
    u";\]": "Wink or smirk",
    u";\^\)": "Wink or smirk",
    u":‑,": "Wink or smirk",
    u";D": "Wink or smirk",
    u":‑P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S": "Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|": "Straight face",
    u":\|": "Straight face",
    u":$": "Embarrassed or blushing",
    u":‑x": "Sealed lips or wearing braces or tongue-tied",
    u":x": "Sealed lips or wearing braces or tongue-tied",
    u":‑#": "Sealed lips or wearing braces or tongue-tied",
    u":#": "Sealed lips or wearing braces or tongue-tied",
    u":‑&": "Sealed lips or wearing braces or tongue-tied",
    u":&": "Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)": "Angel, saint or innocent",
    u"O:\)": "Angel, saint or innocent",
    u"0:‑3": "Angel, saint or innocent",
    u"0:3": "Angel, saint or innocent",
    u"0:‑\)": "Angel, saint or innocent",
    u"0:\)": "Angel, saint or innocent",
    u":‑b": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)": "Angel, saint or innocent",
    u">:‑\)": "Evil or devilish",
    u">:\)": "Evil or devilish",
    u"\}:‑\)": "Evil or devilish",
    u"\}:\)": "Evil or devilish",
    u"3:‑\)": "Evil or devilish",
    u"3:\)": "Evil or devilish",
    u">;\)": "Evil or devilish",
    u"\|;‑\)": "Cool",
    u"\|‑O": "Bored",
    u":‑J": "Tongue-in-cheek",
    u"#‑\)": "Party all night",
    u"%‑\)": "Drunk or confused",
    u"%\)": "Drunk or confused",
    u":-###..": "Being sick",
    u":###..": "Being sick",
    u"<:‑\|": "Dump",
    u"\(>_<\)": "Troubled",
    u"\(>_<\)>": "Troubled",
    u"\(';'\)": "Baby",
    u"\(\^\^>``": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz": "Sleeping",
    u"\(\^_-\)": "Wink",
    u"\(\(\+_\+\)\)": "Confused",
    u"\(\+o\+\)": "Confused",
    u"\(o\|o\)": "Ultraman",
    u"\^_\^": "Joyful",
    u"\(\^_\^\)/": "Joyful",
    u"\(\^O\^\)／": "Joyful",
    u"\(\^o\^\)／": "Joyful",
    u"\(__\)": "Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_": "Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>": "Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>": "Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m": "Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m": "Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)": "Sad or Crying",
    u"\(/_;\)": "Sad or Crying",
    u"\(T_T\) \(;_;\)": "Sad or Crying",
    u"\(;_;": "Sad of Crying",
    u"\(;_:\)": "Sad or Crying",
    u"\(;O;\)": "Sad or Crying",
    u"\(:_;\)": "Sad or Crying",
    u"\(ToT\)": "Sad or Crying",
    u";_;": "Sad or Crying",
    u";-;": "Sad or Crying",
    u";n;": "Sad or Crying",
    u";;": "Sad or Crying",
    u"Q\.Q": "Sad or Crying",
    u"T\.T": "Sad or Crying",
    u"QQ": "Sad or Crying",
    u"Q_Q": "Sad or Crying",
    u"\(-\.-\)": "Shame",
    u"\(-_-\)": "Shame",
    u"\(一一\)": "Shame",
    u"\(；一_一\)": "Shame",
    u"\(=_=\)": "Tired",
    u"\(=\^\·\^=\)": "cat",
    u"\(=\^\·\·\^=\)": "cat",
    u"=_\^=	": "cat",
    u"\(\.\.\)": "Looking down",
    u"\(\._\.\)": "Looking down",
    u"\^m\^": "Giggling with hand covering mouth",
    u"\(\・\・?": "Confusion",
    u"\(?_?\)": "Confusion",
    u">\^_\^<": "Normal Laugh",
    u"<\^!\^>": "Normal Laugh",
    u"\^/\^": "Normal Laugh",
    u"\（\*\^_\^\*）": "Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)": "Normal Laugh",
    u"\(^\^\)": "Normal Laugh",
    u"\(\^\.\^\)": "Normal Laugh",
    u"\(\^_\^\.\)": "Normal Laugh",
    u"\(\^_\^\)": "Normal Laugh",
    u"\(\^\^\)": "Normal Laugh",
    u"\(\^J\^\)": "Normal Laugh",
    u"\(\*\^\.\^\*\)": "Normal Laugh",
    u"\(\^—\^\）": "Normal Laugh",
    u"\(#\^\.\^#\)": "Normal Laugh",
    u"\（\^—\^\）": "Waving",
    u"\(;_;\)/~~~": "Waving",
    u"\(\^\.\^\)/~~~": "Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~": "Waving",
    u"\(T_T\)/~~~": "Waving",
    u"\(ToT\)/~~~": "Waving",
    u"\(\*\^0\^\*\)": "Excited",
    u"\(\*_\*\)": "Amazed",
    u"\(\*_\*;": "Amazed",
    u"\(\+_\+\) \(@_@\)": "Amazed",
    u"\(\*\^\^\)v": "Laughing,Cheerful",
    u"\(\^_\^\)v": "Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)": "Headphones,Listening to music",
    u'\(-"-\)': "Worried",
    u"\(ーー;\)": "Worried",
    u"\(\^0_0\^\)": "Eyeglasses",
    u"\(\＾ｖ\＾\)": "Happy",
    u"\(\＾ｕ\＾\)": "Happy",
    u"\(\^\)o\(\^\)": "Happy",
    u"\(\^O\^\)": "Happy",
    u"\(\^o\^\)": "Happy",
    u"\)\^o\^\(": "Happy",
    u":O o_O": "Surprised",
    u"o_0": "Surprised",
    u"o\.O": "Surpised",
    u"\(o\.o\)": "Surprised",
    u"oO": "Surprised",
    u"\(\*￣m￣\)": "Dissatisfied",
    u"\(‘A`\)": "Snubbed or Deflated"
}

In [678]:
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'(' + emot + ')', "_".join(EMOTICONS[emot].replace(",", "").split()), text)
    return text


text = "Hello :-) :-)"
convert_emoticons(text)

'Hello Happy_face_smiley Happy_face_smiley'

In [679]:
#TODO EMOJI replace with pytyhon emoji library

## Remove urls and HTML

In [680]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [681]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)


## Tokenizacja

In [682]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()


def tokenize_tweets(text):
    return tweet_tokenizer.tokenize(text)

df["text_tokenize"] = df["text_chat_conversion"].apply(lambda text: tokenize_tweets(text))
df["text_tokenize_preprocessed"] = df["text_preprocessed"].apply(lambda text: tokenize_tweets(text))

In [683]:
df.T

,0,1,2
text,"The cars drive, fast and erratically",the car dsrive slow?,The bike ride very fast and erratically!
text_chat_conversion,The cars drive fast and erratically,the car drive slow,The bike ride very fast and erratically
correct_spellings,The cars drive fast and erratically,the car drive slow,The bike ride very fast and erratically
text_punctation_remove,The cars drive fast and erratically,the car drive slow,The bike ride very fast and erratically
text_lower,the cars drive fast and erratically,the car drive slow,the bike ride very fast and erratically
text_stopwords_removal,cars drive fast erratically,car drive slow,bike ride fast erratically
text_stemmed,car drive fast errat,car drive slow,bike ride fast errat
text_preprocessed,car drive fast erratically,car drive slow,bike ride fast erratically
text_lemmatized_with_pos,The car drive fast and erratically,the car drive slow,The bike ride very fast and erratically
text_tokenize,"[The, cars, drive, fast, and, erratically]","[the, car, drive, slow]","[The, bike, ride, very, fast, and, erratically]"


# ANALIZA DANYCH

## Bag of Words

In [684]:
vectorizer = CountVectorizer(lowercase=False)
vectorizer.fit(df["text_preprocessed"])
print(sorted(vectorizer.vocabulary_))
vector = vectorizer.transform(df["text_preprocessed"])

['bike', 'car', 'drive', 'erratically', 'fast', 'ride', 'slow']


In [685]:
print(vector.toarray())
pd.DataFrame(vector.toarray(),columns=sorted(vectorizer.vocabulary_))

[[0 1 1 1 1 0 0]
 [0 1 1 0 0 0 1]
 [1 0 0 1 1 1 0]]


,bike,car,drive,erratically,fast,ride,slow
0,0,1,1,1,1,0,0
1,0,1,1,0,0,0,1
2,1,0,0,1,1,1,0


In [686]:
vectorizer.fit(df["text_punctation_remove"])
print(sorted(vectorizer.vocabulary_))
vector = vectorizer.transform(df["text_punctation_remove"])

['The', 'and', 'bike', 'car', 'cars', 'drive', 'erratically', 'fast', 'ride', 'slow', 'the', 'very']


In [687]:
print(vector.toarray())
pd.DataFrame(vector.toarray(),columns=sorted(vectorizer.vocabulary_))

[[1 1 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 1 1 0]
 [1 1 1 0 0 0 1 1 1 0 0 1]]


,The,and,bike,car,cars,drive,erratically,fast,ride,slow,the,very
0,1,1,0,0,1,1,1,1,0,0,0,0
1,0,0,0,1,0,1,0,0,0,1,1,0
2,1,1,1,0,0,0,1,1,1,0,0,1
